In [56]:
import numpy as np
from pandas import read_csv
from pandas import to_datetime
from sklearn.preprocessing import MinMaxScaler
import math

# load the train data
dataframe = read_csv('dataset/train.csv',usecols=[i for i in range(0,211)], engine='python', header=0)
trainDataOriginal = dataframe.values
trainDataOriginal = np.nan_to_num(trainData)
trainData = trainDataOriginal
# load the test data
dataframe = read_csv('dataset/test_2.csv',usecols=[i for i in range(0,146)], engine='python', header=0)
testData = dataframe.values
testData = np.nan_to_num(testData)
testX = testData

In [57]:
#parameters
batch_size = 64
input_dim  = 146
output_dim = 64

In [58]:
#Separate the train data to input and output
trainX = trainData[0:len(trainData),0:146]
trainY = trainData[0:len(trainData),147:211]
# normalize the dataset
scalerInput = MinMaxScaler(feature_range=(0, 1))
scalerOutput = MinMaxScaler(feature_range=(0, 1))

trainX = scalerInput.fit_transform(trainX)
trainY = scalerOutput.fit_transform(trainY)
testX = scalerInput.fit_transform(testX)

#Put it into batches
trainX = np.reshape(trainX,(len(trainX)/batch_size,batch_size,trainX.shape[1]))
trainY = np.reshape(trainY,(len(trainY)/batch_size,batch_size,trainY.shape[1]))
testX  = np.reshape(testX,(len(testX)/batch_size,batch_size,testX.shape[1]))

print trainX.shape
print trainY.shape
print testX.shape

(625, 64, 146)
(625, 64, 64)
(1875, 64, 146)


In [59]:
from keras.activations import linear
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Input
from keras.optimizers import adam
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.layers import Input, Dense

# create the model
Opt = adam(0.1, 0.1, 0.001, 1e-08, 0.0)
model = Sequential()
model.add(LSTM(output_dim, input_shape=(batch_size, input_dim), return_sequences=True))
print model.output_shape
model.add(Dense(output_dim, activation='linear'))
print model.output_shape
#
#Configures the model for training.
model.compile(loss="mean_squared_error",optimizer=Opt)
#Trains the model for a fixed number of epochs (iterations on a dataset).
model.fit(trainX,trainY,batch_size=batch_size,epochs=1)
#Make predictions
testY = model.predict(testX)

(None, 64, 64)
(None, 64, 64)
Epoch 1/1
625/625 [==============================] - 2s - loss: 21.9218     


In [63]:
testY = testY.reshape(len(testData),output_dim)
testY = scalerOutput.inverse_transform(testY)

